## Non-interacting_4-photon_States

This is a project to get all 4-photon states when __two__ entangled photon pairs, __very close__ to each other, incident upon __only one arm__ of a beam splitter (input: 1) and get either transmitted or reflected. Then, we will filter out the symmetric cases (2 transmitted, 2 reflected). 


In [1]:
import numpy as np
from sympy import *
from IPython.display import display, Latex


- `commutative=False` means $H_e \; V_e  \neq V_e \; He$
- `Rational( , )` is from the __sympy__ package, for better display: $0.707 \rightarrow \frac{1}{\sqrt{2}}$
- `.subs()` is a __sympy__ function, for substituing a variable: $H_e \rightarrow \frac{1}{\sqrt{2}}(H_t^e + H_r^e)$
- `simplify()` is a __sympy__ function, foralgebric simplification: $\frac{x^2 + 2x}{x} \rightarrow x+2$ 

In [2]:

# list of symbolic representation
He, Hl, Ve, Vl, Hte, Hre, Htl, Hrl, Vte, Vre, Vtl, Vrl, = symbols('H^e H^l V^e V^l H_t^e H_r^e H_t^l H_r^l V_t^e V_r^e V_t^l V_r^l', commutative=False)

#for simplication in the output display format
sqrt_half = Rational(1,2)**Rational(1,2)

psi_minus_early  = sqrt_half * (He * Ve - Ve * He)
psi_minus_later  = sqrt_half * (Hl * Vl - Vl * Hl)

H_transform_e    = sqrt_half * (Hte + Hre)
V_transform_e    = sqrt_half * (Vte + Vre)

H_transform_l    = sqrt_half * (Htl + Hrl)
V_transform_l    = sqrt_half * (Vtl + Vrl)

psi_bs_applied_e = psi_minus_early.subs({He: H_transform_e, Ve: V_transform_e})
psi_bs_applied_l = psi_minus_later.subs({Hl: H_transform_l, Vl: V_transform_l})

# Expand and simplify
psi_bs_applied_e = simplify(expand(psi_bs_applied_e))
psi_bs_applied_l = simplify(expand(psi_bs_applied_l))

# Display result
display(Latex(r'$\psi^-_{\text{early,transformed}} = ' + latex(psi_bs_applied_e) + r'$'))
display(Latex(r'$\psi^-_{\text{later,transformed}} = ' + latex(psi_bs_applied_l) + r'$'))

psi_minus_output_bs1 = psi_bs_applied_e * psi_bs_applied_l
psi_minus_output_bs1 = simplify(expand(psi_minus_output_bs1))
display(Latex(r'$\psi^-_{\text{BS1}} = ' + latex(psi_minus_output_bs1) + r'$'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Filtering the symmetry outputs

We got 64 states. 

- `str()` is a built in __python__ package function converts the sympy symbols back to string, consistes of letters: $H_r^e \rightarrow$ H_r^e
- `expand()` is a __sympy__ function to expand __products, powers, distributions__ : $(a+b)^2 \rightarrow a^2 + 2ab + b^2$ [kinda opposite to `simplify()`]
- `.args` belongs to __sympy__, gives you the pieces being added
- `Add( , )` is a __sympy__ function, corresponds to symbolic addition expression class: `Add(x, y, z) = x+y+z`
- `isinstance( , )` is a built in __python__ library function, that can work with another function from another library. Here, it is in the format: `isinstance(terms, Add)`. This first checks if there are elements in`terms` are related by __sum__. If so, _each_ of these are treated as list elements. If there is no `+` found, the whole element would be listed as a single element (special case). 

In [3]:

import re as regex

# Ensure fully expanded
terms      = expand(psi_minus_output_bs1)
term_list  = terms.args if isinstance(terms, Add) else [terms]

# Count '_r^' and '_t^' using string methods (faster & simpler)
def even_r_and_t_filter(τϵρμ):
    τϵρμ_str = str(τϵρμ)
    return τϵρμ_str.count('_r^') == 2 and τϵρμ_str.count('_t^') == 2

# Filter terms: keep only those with 2 reflected + 2 transmitted photons
# filtered_terms = [term for term in term_list if even_r_and_t_filter(term)]

filtered_terms = []
for i in term_list:
    if even_r_and_t_filter(i):
        filtered_terms.append(i)

# Recombine and simplify
even_terms = simplify(sum(filtered_terms))

# Display
display(Latex(r'$\psi^-_{\text{BS1, even\ terms}} = ' + latex(even_terms) + r'$'))


<IPython.core.display.Latex object>

## Let's save the data in a CSV file using `pandas`

- `.as_coeff_Mul()` is a __sympy__ library function. If `a = 3*x**2`, it works like: `b, c = a.as_ordered_terms() = [3, x**2]`

In [4]:

import pandas as pd

# Split even_terms into individual terms
term_data = []

for i in filtered_terms:
    coeff, expr = i.as_coeff_Mul()  # Separate coefficient and symbolic part
    term_data.append((str(expr), coeff))  # Store as (term string, coeff as Rational)

# Create DataFrame
df = pd.DataFrame(term_data, columns=['Term', 'Coefficient'])

# Save as CSV
df.to_csv("bs1_even_terms.csv", index=False)
print("CSV saved as 'bs1_even_terms.csv'")


CSV saved as 'bs1_even_terms.csv'
